# Rockset | Rockset

> [Rockset](https://rockset.com/)は、クラウド向けに構築されたリアルタイム検索および分析データベースです。Rocksetは、スケールに応じた低遅延で高並行性の検索クエリを提供するために、効率的なベクトル埋め込みのストアを備えた[Converged Index™](https://rockset.com/blog/converged-indexing-the-secret-sauce-behind-rocksets-fast-queries/)を使用しています。Rocksetはメタデータのフィルタリングを完全にサポートし、絶えず更新されるストリーミングデータのリアルタイム取り込みを処理します。
>
> > [Rockset](https://rockset.com/) is a real-time search and analytics database built for the cloud. Rockset uses a [Converged Index™](https://rockset.com/blog/converged-indexing-the-secret-sauce-behind-rocksets-fast-queries/) with an efficient store for vector embeddings to serve low latency, high concurrency search queries at scale. Rockset has full support for metadata filtering and  handles real-time ingestion for constantly updating, streaming data.

このノートブックは、LangChainで`Rockset`をベクターストアとして使用する方法を示しています。始める前に、`Rockset`アカウントへのアクセスと利用可能なAPIキーを持っていることを確認してください。[今すぐ無料トライアルを開始してください。](https://rockset.com/create/)

> This notebook demonstrates how to use `Rockset` as a vector store in LangChain. Before getting started, make sure you have access to a `Rockset` account and an API key available. [Start your free trial today.](https://rockset.com/create/)




## Setting Up Your Environment[](https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/rockset#setting-up-environment) | 環境の設定[](https://python.langchain.com/docs/modules/data_connection/vectorstores/integrations/rockset#setting-up-environment)

1. `Rockset`コンソールを利用して、Write APIをソースとする[コレクション](https://rockset.com/docs/collections/)を作成します。このウォークスルーでは、`langchain_demo`という名前のコレクションを作成します。

   > Leverage the `Rockset` console to create a [collection](https://rockset.com/docs/collections/) with the Write API as your source. In this walkthrough, we create a collection named `langchain_demo`.

   以下の[データ取り込み変換](https://rockset.com/docs/ingest-transformation/)を設定して、あなたのembeddingsフィールドにマークを付け、パフォーマンスとストレージの最適化を活用するようにしてください：

   > Configure the following [ingest transformation](https://rockset.com/docs/ingest-transformation/) to mark your embeddings field and take advantage of performance and storage optimizations:

   この例では、OpenAIの`text-embedding-ada-002`を使用しました（ベクトル埋め込みの長さ＝1536です）。

   > (We used OpenAI `text-embedding-ada-002` for this examples, where #length\_of\_vector\_embedding = 1536)




```
SELECT _input.* EXCEPT(_meta), 
VECTOR_ENFORCE(_input.description_embedding, #length_of_vector_embedding, 'float') as description_embedding 
FROM _input
```

2. コレクションを作成した後、コンソールを使用して[APIキー](https://rockset.com/docs/iam/#users-api-keys-and-roles)を取得してください。このノートブックでは、`Oregon(us-west-2)`リージョンを使用していることを前提としています。

   > After creating your collection, use the console to retrieve an [API key](https://rockset.com/docs/iam/#users-api-keys-and-roles). For the purpose of this notebook, we assume you are using the `Oregon(us-west-2)` region.


3. `Rockset`と直接通信できるようにするため、[rockset-python-client](https://github.com/rockset/rockset-python-client)をインストールしてLangChainを有効にしてください。

   > Install the [rockset-python-client](https://github.com/rockset/rockset-python-client) to enable LangChain to communicate directly with `Rockset`.





In [ ]:
!pip install rockset

## LangChain Tutorial | LangChainチュートリアル

ご自身のPythonノートブックで手順に従い、ベクトル埋め込みを生成してRocksetに保存しましょう。そして、検索クエリに類似するドキュメントを探すためにRocksetの使用を開始しましょう。

> Follow along in your own Python notebook to generate and store vector embeddings in Rockset.
> Start using Rockset to search for documents similar to your search queries.

### 1. Define Key Variables | 1. 主要変数の定義




In [ ]:
import os

import rockset

ROCKSET_API_KEY = os.environ.get(
    "ROCKSET_API_KEY"
)  # Verify ROCKSET_API_KEY environment variable
ROCKSET_API_SERVER = rockset.Regions.usw2a1  # Verify Rockset region
rockset_client = rockset.RocksetClient(ROCKSET_API_SERVER, ROCKSET_API_KEY)

COLLECTION_NAME = "langchain_demo"
TEXT_KEY = "description"
EMBEDDING_KEY = "description_embedding"

### 2. Prepare Documents | 2. 文書の準備




In [ ]:
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Rockset

loader = TextLoader("../../modules/state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

### 3. Insert Documents | 3. ドキュメントの挿入




In [ ]:
embeddings = OpenAIEmbeddings()  # Verify OPENAI_API_KEY environment variable

docsearch = Rockset(
    client=rockset_client,
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    text_key=TEXT_KEY,
    embedding_key=EMBEDDING_KEY,
)

ids = docsearch.add_texts(
    texts=[d.page_content for d in docs],
    metadatas=[d.metadata for d in docs],
)

### 4. Search for Similar Documents | 4. 類似文書の検索




In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
output = docsearch.similarity_search_with_relevance_scores(
    query, 4, Rockset.DistanceFunction.COSINE_SIM
)
print("output length:", len(output))
for d, dist in output:
    print(dist, d.metadata, d.page_content[:20] + "...")

##
# output length: 4
# 0.764990692109871 {'source': '../../../state_of_the_union.txt'} Madam Speaker, Madam...
# 0.7485416901622112 {'source': '../../../state_of_the_union.txt'} And I’m taking robus...
# 0.7468678973398306 {'source': '../../../state_of_the_union.txt'} And so many families...
# 0.7436231261419488 {'source': '../../../state_of_the_union.txt'} Groups of citizens b...

### 5. Search for Similar Documents with Filtering | 5. フィルタリングを使用した類似ドキュメントの検索




In [ ]:
output = docsearch.similarity_search_with_relevance_scores(
    query,
    4,
    Rockset.DistanceFunction.COSINE_SIM,
    where_str="{} NOT LIKE '%citizens%'".format(TEXT_KEY),
)
print("output length:", len(output))
for d, dist in output:
    print(dist, d.metadata, d.page_content[:20] + "...")

##
# output length: 4
# 0.7651359650263554 {'source': '../../../state_of_the_union.txt'} Madam Speaker, Madam...
# 0.7486265516824893 {'source': '../../../state_of_the_union.txt'} And I’m taking robus...
# 0.7469625542348115 {'source': '../../../state_of_the_union.txt'} And so many families...
# 0.7344177777547739 {'source': '../../../state_of_the_union.txt'} We see the unity amo...

### 6. \[Optional] Delete Inserted Documents | 6. \[オプショナル] 挿入されたドキュメントを削除する

コレクションからドキュメントを削除するには、各ドキュメントに関連付けられた一意のIDが必要です。`Rockset.add_texts()`を使用してドキュメントを挿入する際にIDを定義することができます。そうしない場合、Rocksetは各ドキュメントに対して一意のIDを生成します。いずれにせよ、`Rockset.add_texts()`は挿入されたドキュメントのIDを返します。

> You must have the unique ID associated with each document to delete them from your collection.
> Define IDs when inserting documents with `Rockset.add_texts()`. Rockset will otherwise generate a unique ID for each document. Regardless, `Rockset.add_texts()` returns the IDs of inserted documents.

これらのドキュメントを削除するには、`Rockset.delete_texts()` 関数を使用してください。

> To delete these docs, simply use the `Rockset.delete_texts()` function.




In [ ]:
docsearch.delete_texts(ids)

## Summary | 概要

このチュートリアルでは、`Rockset` コレクションを作成し、OpenAIの埋め込みを含むドキュメントを `挿入` し、メタデータフィルターを使用する場合と使用しない場合の両方で、類似ドキュメントの検索に成功しました。

> In this tutorial, we successfully created a `Rockset` collection, `inserted` documents with  OpenAI embeddings, and searched for similar documents with and without metadata filters.

この分野の今後の更新情報については、https://rockset.com/ をチェックし続けてください。

> Keep an eye on https://rockset.com/ for future updates in this space.


